### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification for Part 1
threshold_num = 1

# word all usage in sent
word_use_num_max = 5
word_use_num_min = 1

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

#### Part 1

In [5]:
def column_word_list(df, column):
    word_list = []
    for i in df[f"{column}"].dropna():
        words = i.split(",")
        for j in words:
            word = j.strip()
            word_list.append(word)
    return word_list

In [6]:
def column_word(df, column_list):
    '''
    column_word(df_pos_tag_group, df_pos_tag_group.columns) \n
    column_word(df_pos_tag_group, ["col_1","col_2"])
    '''
    word_list_all = []
    for i in column_list:
        word_list = column_word_list(df, f"{i}")
        word_list_all.append(word_list)
    return word_list_all

In [7]:
def classification_condition(list_x, count_condition):
    '''
    from collections import Counter needed
    
    '''
    dict_list_count = Counter(list_x)
    count_list = list(dict_list_count.values())

    if any([True if i>count_condition else False for i in count_list]):
        return "Fail"
    else:
        return "Pass"

    #return any([True if i>count_condition else False for i in count_list])  

In [8]:
df_pos_tag_group = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/\
Data/{lang_folder.capitalize()}/{file}.xlsx", sheet_name=f"{sheets}")
df_pos_tag_group

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8
0,adam,başka,al,ama,"ben, beni, benimle, bana, bence, benim",artık,aynı,dostum,"mı, mısın"
1,anne,belki,"bak, bakalım",çok,"bir, biri",asla,biraz,efendim,"misin, mi"
2,baba,bile,"biliyorum, biliyorsun, bilmiyorum, biliyor",da,"bizi, bizim, bize, biz",birkaç,büyük,hadi,mü
3,"gerçek, gerçekten",birlikte,"değil, değilim",daha,"bu, bunun, buna, bunu, burada, buraya",bugün,doğru,haydi,"mu, musun"
4,iş,böyle,"demek, de",diye,"o, onu, onları, ona, onun, onlar, onunla, orada",dakika,en,hey,nasıl
5,izin,bütün,dur,evet,"seni, seninle, senin, sen, sana",devam,güzel,"iş, işte","ne, neler"
6,kız,çünkü,"ederim, et",gibi,"size, sizi, siz",gece,harika,işte,neden
7,önemli,eğer,"gel, geldi, geliyor",hayır,"şu, şunu",gün,iki,lütfen,nerede
8,özür,fazla,git,için,NaN,hala,iyi,merhaba,NaN
9,para,gerek,ister,ile,NaN,hemen,kötü,oh,NaN


In [9]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.capitalize()}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [10]:
column_word_list_all = column_word(df_pos_tag_group, df_pos_tag_group.columns)

In [11]:
result_list = []
#for i in df_youtube_sent_select["search_string"]:
for i in range(len(df_youtube_sent_result)):
    #words = word_tokenize(i)
    search = df_youtube_sent_result.loc[i,"search_string"]
    start=df_youtube_sent_result.loc[i,"start_time"]
    end=df_youtube_sent_result.loc[i,"end_time"]
    sent=df_youtube_sent_result.loc[i,"sentence"]
    id = df_youtube_sent_result.loc[i,"video_id"]
    id_url = df_youtube_sent_result.loc[i,"video_url"]
    words = word_tokenize(search)
    list_var = []
    for j in words:
        a = 0        
        while a < len(column_word_list_all):            
            if j in column_word_list_all[a]:
                list_var.append(a)
            else:
                pass                
            a+=1
    result_list.append([search,start,end,sent,id,id_url,list_var])

In [12]:
df_youtube_sent_class = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","classification"])
df_youtube_sent_class

,search_string,start_time,end_time,sentence,video_id,video_url,classification
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]"
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]"
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]"
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]"
...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]"
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]"
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]"
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[4, 8, 4, 8]"


In [13]:
df_youtube_sent_class["condition"] = df_youtube_sent_class["classification"].apply(lambda x: classification_condition(x, threshold_num))
df_youtube_sent_class

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[0, 2, 3, 7, 3, 5, 1, 5, 7, 1, 5, 5]",Fail
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[2, 1, 1, 1, 2, 1, 3, 1, 2, 3, 1, 2, 8, 2]",Fail
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[7, 1, 4, 1, 2, 8, 7, 3, 4, 1, 2, 8, 7, 8, 8]",Fail
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[4, 3, 0, 2, 6, 1, 5, 4, 2, 4, 0, 5]",Fail
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[2, 2, 4, 3, 2, 2, 4, 3, 2, 3, 5, 0, 7, 4, 3]",Fail
...,...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[4, 8, 4, 8]",Fail
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[4, 8, 3, 4]",Fail
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[4, 8, 8, 4]",Fail
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[4, 8, 4, 8]",Fail


In [14]:
#df_youtube_sent_class.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_All.xlsx", index=False)

In [15]:
df_youtube_sent_class_select = df_youtube_sent_class[df_youtube_sent_class["condition"] == "Pass"]
df_youtube_sent_class_select.reset_index(drop=True, inplace=True)
df_youtube_sent_class_select

,search_string,start_time,end_time,sentence,video_id,video_url,classification,condition
0,ya gerçekten hiç gerek yok ben çok iyi,2146,2148,ya gerçekten hiç gerek yok ben çok iyi bakacağ...,qV3xntSmr1o,https://www.youtube.com/watch?v=qV3xntSmr1o&t=...,"[7, 0, 5, 1, 2, 4, 3, 6]",Pass
1,mu daha önce seninle birlikte olduğunu,158,160,bahar bilmiyor mu daha önce seninle birlikte o...,qVk4p_4pwv8,https://www.youtube.com/watch?v=qVk4p_4pwv8&t=...,"[8, 3, 5, 4, 1, 2]",Pass
2,çünkü burada neler olduğunu gerçekten,1450,1453,çünkü burada neler olduğunu gerçekten bilmek i...,OmvrQaXfrZQ,https://www.youtube.com/watch?v=OmvrQaXfrZQ&t=...,"[1, 4, 8, 2, 0]",Pass
3,gerçekten çok teşekkür ederim o zaman,391,396,meral hanım gerçekten çok teşekkür ederim o zaman,fLgorIU0es4,https://www.youtube.com/watch?v=fLgorIU0es4&t=...,"[0, 3, 7, 2, 4, 5]",Pass
4,geliyor adam gibi bir şeyler şimdi ne,1192,1197,iyi olsa bile getirmiyorsun ne yapacağız bugün...,JyzAOR0GTs4,https://www.youtube.com/watch?v=JyzAOR0GTs4&t=...,"[2, 0, 3, 4, 1, 5, 8]",Pass
...,...,...,...,...,...,...,...,...
32260,ya da hiç o,4949,4950,ya da hiç o riske girme direkt git buradan ya,dHohXDBK3KI,NaN,"[7, 3, 5, 4]",Pass
32261,o kız mı da,142,145,o kız mı da belirliyorum,oDt7oJ_raio,NaN,"[4, 0, 8, 3]",Pass
32262,var o da ne,670,673,burası çünkü bir dükkan canım ya kaşgarlı türk...,y4N3us_VXRA,NaN,"[2, 4, 3, 8]",Pass
32263,o ne ki ya,3512,3513,o ne ki ya kısa oruç öğlene kadar,KHyLRPII8ow,NaN,"[4, 8, 3, 7]",Pass


In [16]:
#df_youtube_sent_class_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_{threshold_num}_Classification_Select.xlsx", index=False)

#### Part 2

In [5]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [6]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [7]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [8]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...
...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN


In [9]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[87, 197, 9, 100, 9, 35, 30, 35, 60, 30, 35, 106]"
1,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
4,yok yok o kadar yok yok o kadar yok ama şimdi ...,2162,2168,yok yok o kadar yok yok o kadar yok ama şimdi ...,DbsR_Cccadg,https://www.youtube.com/watch?v=DbsR_Cccadg&t=...,"[22, 22, 6, 20, 22, 22, 6, 20, 22, 10, 43, 183..."
...,...,...,...,...,...,...,...
120259,o ne o ne,5133,5135,o ne o ne,1s60RPDoaNU,NaN,"[6, 2, 6, 2]"
120260,o ne ki o,1653,1654,nicki o ne ki o,K8fgPqaaJrA,NaN,"[6, 2, 33, 6]"
120261,o ne ne o,4250,4252,o ne ne o,bIppytTQ0aI,NaN,"[6, 2, 2, 6]"
120262,o mu o mu,4062,4064,o mu o mu,8cU1k2ylhF4,NaN,"[6, 47, 6, 47]"


In [10]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [11]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
4,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
...,...,...,...,...,...,...,...
36337,mi o da ne,444,445,kabayel mi o da ne,UxIEpobTj6M,NaN,"[5, 6, 15, 2]"
36338,var o da o,416,418,bu yapılarda çok güzel taşları görünüşü bu köy...,mx73zmdFMys,NaN,"[11, 6, 15, 6]"
36339,o da o da,515,517,o da gitti o da o da ikimizde ya işte anlatırk...,bc8xOmoiZm8,NaN,"[6, 15, 6, 15]"
36340,o ne o ne,188,190,o ne o ne,uBDsQXn1DgQ,NaN,"[6, 2, 6, 2]"


In [12]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [13]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [14]:
#word_num_index_dict

In [15]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [16]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[162, 163, 100, 101, 164, 165, 136, 140, 145, 82, 178, 151, 185, 122, 59, 92]

In [17]:
#Counter(index_list)

In [18]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
...,...,...,...,...,...,...,...
116,iki saat üç saat,353,354,iki saat üç saat geçti,EKojB84HQfg,NaN,"[110, 195, 171, 195]"
117,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,NaN,"[118, 71, 78, 118]"
118,al sana işte al,150,152,al sana işte al,7Pt6Kj7ZGxc,NaN,"[169, 32, 108, 169]"
119,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,NaN,"[48, 81, 48, 81]"


In [31]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [4]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel("threegram_selected.xlsx")
df_selected_file

,threegram
0,adam yine geldi
1,al bakalım dostum
2,al şunu al
3,al şunu da
4,ama asla olmaz
...,...
485,yoksa kötü mü
486,yoksa sizi de
487,zaman bugün mü
488,zaman seninle olsun


In [5]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [6]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,bu,38
1,değil,26
2,ne,24
3,ama,22
4,mi,21
...,...,...
200,düşünüyorsun,1
201,eve,1
202,belki,1
203,büyük,1


In [ ]:
df_shared_word_manuel_count.to_excel("Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised_Word_Count.xlsx", index=False)

In [8]:
# Option *****
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all = df_word_all.head(200)
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
set_threegram_word = set(df_shared_word_manuel_count["word"])
set_word = set(df_word_all["word"])

In [10]:
set_threegram_word.difference(set_word)
# Option ***

{'buradan', 'dilerim', 'düşünüyorsun', 'eve', 'gidelim', 'ilgili', 'kazanmak'}

##### Not In Sent And Selected Word List

In [26]:
lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [27]:
word_start = 0
word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [28]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [29]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['özür',
 'mü',
 'teşekkürler',
 'yeni',
 'dostum',
 'selam',
 'hakkında',
 'izin',
 'git',
 'hey',
 'para',
 'mısın',
 'pekala',
 'üzgünüm',
 'bile',
 'tüm']

In [30]:
set_not_in_sent_word = set(not_in_sent_word)

In [31]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [32]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'gece', 'pekala'}

In [34]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['dostum',
 'para',
 'yeni',
 'git',
 'hey',
 'mısın',
 'üzgünüm',
 'izin',
 'gece',
 'özür',
 'pekala',
 'mü',
 'hakkında',
 'teşekkürler',
 'bile',
 'tüm',
 'selam']

In [35]:
len(not_in_sent_selected_word_list)

17

In [36]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
2,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
3,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
4,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...
...,...,...,...,...,...,...
36337,mi o da ne,444,445,kabayel mi o da ne,UxIEpobTj6M,NaN
36338,var o da o,416,418,bu yapılarda çok güzel taşları görünüşü bu köy...,mx73zmdFMys,NaN
36339,o da o da,515,517,o da gitti o da o da ikimizde ya işte anlatırk...,bc8xOmoiZm8,NaN
36340,o ne o ne,188,190,o ne o ne,uBDsQXn1DgQ,NaN


In [37]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [38]:
not_in_sent_selected_word_list_result = word_in_wordgroup_simple(not_in_sent_selected_word_list, df_youtube_sent_select, "search_string", 5)
not_in_sent_selected_word_list_result

,word,search_string
0,dostum,sorun değil onlar değil mi dostum
1,dostum,olur mu öyle şey o benim dostum
2,dostum,evet dostum o adam iyi ki bizi
3,dostum,zaman o benim dostum her zaman
4,dostum,evet dostum o adam iyi ki bizi
...,...,...
75,selam,için selam yok mu teşekkür ederim senin seninle
76,selam,selam söyle çok fazla çok güzel bir
77,selam,ben aslında buraya sadece selam
78,selam,bakalım burada ne kadar selam


In [39]:
not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
not_in_sent_selected_word_list_merge

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,dostum,sorun değil onlar değil mi dostum,77,80,tugay özellikle yaş ilerledikçe daha da artaca...,yb6NCcmsSMo,https://www.youtube.com/watch?v=yb6NCcmsSMo&t=77s
1,dostum,olur mu öyle şey o benim dostum,127,129,olur mu öyle şey o benim dostum,EyxjFfK2GWA,https://www.youtube.com/watch?v=EyxjFfK2GWA&t=...
2,dostum,evet dostum o adam iyi ki bizi,32,35,evet dostum o adam iyi ki bizi seçti,w_F_4UwdR0A,https://www.youtube.com/watch?v=w_F_4UwdR0A&t=32s
3,dostum,evet dostum o adam iyi ki bizi,28,31,evet dostum o adam iyi ki bizi seçti,ZMoMD8tmBZ8,https://www.youtube.com/watch?v=ZMoMD8tmBZ8&t=28s
4,dostum,zaman o benim dostum her zaman,434,436,yola çıktığım zaman o benim dostum her zaman y...,aKhqwoPQ9W8,https://www.youtube.com/watch?v=aKhqwoPQ9W8&t=...
...,...,...,...,...,...,...,...
85,selam,için selam yok mu teşekkür ederim senin seninle,121,127,korkutuyorum olduğu için selam yok mu teşekkür...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
86,selam,selam söyle çok fazla çok güzel bir,669,682,selam söyle çok fazla çok güzel bir tane duyu ...,0mFGLjxTJ1I,https://www.youtube.com/watch?v=0mFGLjxTJ1I&t=...
87,selam,ben aslında buraya sadece selam,73,75,ben aslında buraya sadece selam getirmeye gelm...,FqKLYM0W2D4,https://www.youtube.com/watch?v=FqKLYM0W2D4&t=73s
88,selam,bakalım burada ne kadar selam,659,662,bunun fiyatını ben hatırlıyorum bakalım burada...,o3oQATu9B9A,https://www.youtube.com/watch?v=o3oQATu9B9A&t=...


In [40]:
not_in_sent_selected_word_list_merge.word.nunique()

16

In [41]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_merge["word"].to_list())

In [42]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

{'pekala'}

In [43]:
word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

,word,search_string
0,gece,ben seni değil bir gece bir gün bir saat bir d...
1,gece,burada bu gece tam olarak ne oldu
2,gece,o gece neler olduğunu bilmiyorum
3,gece,ne oldu siz gece gece böyle bir
4,gece,gece neler olduğunu bana şimdi


In [53]:
not_in_sent_selected_word_list_merge.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [50]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Turkish_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised_Word_Count.xlsx']

In [51]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [52]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Custom Pattern

In [ ]:
def classification_order(classification_list, target_pattern_list):
    '''
    classification_order(classification_list, target_pattern_list)
    classification_list and target_pattern_list are list
    '''
    if [x for x in classification_list] == [y for y in target_pattern_list]:
        return "pattern is equal"
    else:
        return "pattern in not equal"

In [ ]:
target_pattern = [7,1,4,1,2,8,7,3,4,1,2,8,7,8,8]

In [ ]:
df_youtube_sent_class["pattern_result"] = df_youtube_sent_class["classification"].apply(lambda x: classification_order(x, target_pattern))
df_youtube_sent_class

In [ ]:
df_youtube_sent_class[df_youtube_sent_class["test"] == "pattern_result"]

In [ ]:
df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Custom_Pattern.xlsx", index=False)

#### Check Selected Result

In [44]:
lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [45]:
word_start = 0
word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [46]:
selected_word_list = df_word_select["word"].to_list()

In [47]:
df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
4,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
...,...,...,...,...,...,...,...
102,gerçekten çok güzel oldu gerçekten harika bir,373,378,yla döküyoruz bu şekilde kıvamları gerçekten ç...,oWLfHDt7KLg,https://www.youtube.com/watch?v=oWLfHDt7KLg&t=...,NaN
103,yani biraz bugün biraz üzgünüm neden,433,437,için çok semih yani biraz bugün biraz üzgünüm ...,DZmXkheI80A,https://www.youtube.com/watch?v=DZmXkheI80A&t=...,NaN
104,söyle bakalım benimle şimdi geliyor musun,207,211,ee söyle bakalım benimle şimdi geliyor musun,9RP0Faoo2rw,https://www.youtube.com/watch?v=9RP0Faoo2rw&t=...,NaN
105,çok teşekkür ederim ya ne demek ben teşekkür e...,649,651,çok tatlısın ömerciğim çok teşekkür ederim ya ...,9uEGE4v9ROM,https://www.youtube.com/watch?v=9uEGE4v9ROM&t=...,NaN


In [48]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [54]:
df_word_in_selected = word_in_wordgroup_simple(selected_word_list, df_selected_file, "search_string", 20)
df_word_in_selected

,word,search_string
0,bir,ya böyle bir şey olabilir mi ya ama bir şey ol...
1,bir,bir şeyi ben böyle istiyorum ve bu şekilde ola...
2,bir,nasıl bir şekilde olduğunu ve neden o şekilde ...
3,bir,için artık değil misin gerek yok bunun için si...
4,bir,bir daha baba tamam lütfen ama ya tamam
...,...,...
625,onunla,biraz sonra size onunla ya ona
626,yapıyorsun,misin hayır peki demek ki nerede yapıyorsun
627,neler,bakalım son olarak neler var neler
628,neler,git de bakalım orada neler oluyor


In [59]:
df_word_in_selected.groupby(["search_string"]).count()

,word
search_string,
al bakalım bu senin gerçekten çok,6
ama ben onun hakkında hiçbir şey,6
aslında benim benim de aslında biraz öyle oldu,6
aslında seninle birlikte orada,4
aynı şekilde burada da var değil mi aynı şekilde burada var,7
...,...
önce tek sonra iki orada üç,6
önemli olduğunu biliyorum o yüzden,5
öyle olmak mı daha iyi böyle olmak mı daha bilmiyorum,7
